#### 그림, 실습코드 등 학습자료 출처 : https://datascienceschool.net

#### 1. 추천시스템 개요

사용자 아이디 카테고리 입력을 받아서 상품 카테고리를 출력하는 classification 문제 같지만 이렇게 하면 풀리지가 않는다. 왜냐하면 인풋이 달랑 하나이기 때문이다. 그런데 인풋이(사용자 아이디) 어마어마하게 많고 아웃풋(상품 카테고리)도 어마어마하게 많다. 

또한 실제로 상품 카테고리를 아웃풋으로 하나만 출력하지 않는다. 다시말해서 추천을 할때 하나만 추천하는 것이 아니라 여러개를 추천한다. 현실적으로는 얘는 사실 카테고리 값을 예측하는 것이 아니라 실제로는 이 유저가 특정아이템을 좋아할 rating을 예측을 하는 것인데 그 예측모델이 뭐냐면 모든 아이템의 평점이 나와야 한다. 예를들어서 1억개의 아이템이 있는데 유저아이디를 하나 입력하면 예측문제를 1억개를 풀어야 한다. 1번 아이템의 평점, 2번 아이템의 평점..., 1억번째 아이템의 평점 이런식으로 풀어야 한다. 그래서 엄청나게 많은 세트의 회귀분석문제를 한꺼번에 푼다고 보면 된다.

근데 이것도 역시 회귀분석 문제라고 단정하기 어려운게 인풋 클래스가 너무 많은게 문제다. 옛날에 나왔던 뭔가 데이터가 있어야 하는데 적은 경우가 많아서 쉽지 않은 문제다. 얼핏보면 분류문제 같지만 안을 들여다보면 회귀분석 문제이고 회귀분석 문제지만 기존의 회귀분석 문제를 푸는 방법론을 조금 다르게 접근해야 한다.

우리가 최종적으로 추천을 하려면 어떤 유저가 들어가서 어떤 아이템을 집어넣으면 평점이 얼마나온다 라는 것을 예측해야 한다. 두개의 카테고리 인풋이 들어가면 한개의 리얼값이 나온다.

- 추천시스템 모델에 반영하기 위한 위한 피벗테이블

이렇게 변환해주면 평점행렬이 나오게 된다. 그러면  어떤 유저가 들어가서 어떤 아이템을 집어넣게 되면 평점이 얼마가 나온다

빈공간은 물건을 안샀다던지 물건을 샀는데도 평점을 안매긴건지는 모르는 것이나 어쨌든 이런 빈공간이 대부분일 것이다.

그래서 우리가 prediction 모델을 만든다는 것은 결국에는 여기의 빈칸을 채우는 것이다.

<img src="0.jpg" width="1000" />

- rate matrix의 빈칸을 채우기 위한 추천시스템 개요

<img src="1.jpg" width="1000" />

#### 2. Baseline model

회귀분석모델에서 카테고리 값 두개만 입력변수다라고 했을때 쓰는 가장 기본적인 모델이 baseline model이다. 

아래 rui헷을 선형회귀모델 형식으로 바꾸면 곧 아래식과 같다.

yui헷 = W0 + WuXu(유저에 대한 카테고리 변수) + WiXi(아이템에 대한 카테고리 변수)



카테고리 값은 더미변수로 인코딩이 되어야한다 그래서 유저개수에 해당하는 u1 u2 u3 .. 이런식으로 유저개수만큼의 어마어마하게 많은 더미변수로 확장이 된다. i1 i2 i3 ... 로 아이템 개수만큼 어마어마하게 많은 더미변수로 확장이 된다.


베이스라인은 그러나 x행렬에 대해서 covariance matrix 만들고 XTX 행렬 만들고 걔의 인버스를 구해서 XT에다가 곱해야 한다. 그런데 얘는 인버스 행렬을 구할 수 없다. 너무 크기 때문이다.

그렇기 때문에 우리가 할 수 있는 것은 SGD 방법밖에 없다. W를 한스텝 한스텝 조금씩 최적점에 다가가는 방법밖에 없다. 그때 error function은 RSS가 된다.

이렇게 푸는 것이 베이스라인의 기본이다. 단 RSS를 할때도 우리가 가지고 있는게 답이 모든게 있는 것이 아니라 일부만 있기 때문에 같고 있는 것만이라도 RSS를 최소화한다.

<img src="2.jpg" width="1000" />

- surprise 패키지를 이용한 파이썬 코드 실습

<img src="3.jpg" width="1000" />

In [72]:
import surprise

data = surprise.Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rate", "id"])
print(df.shape)
df.head(10)

(100000, 4)


,user,item,rate,id
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596
5,298,474,4.0,884182806
6,115,265,2.0,881171488
7,253,465,5.0,891628467
8,305,451,3.0,886324817
9,6,86,3.0,883603013


In [69]:
from surprise import Reader
from surprise import Dataset

reader = Reader(rating_scale=(0,5))
dataframe = Dataset.load_from_df(df[["user", "item", "rate"]],reader)
# loda_from_df에 들어가는 데이터 프레임 컬럼 파라미터는 
# 순서대로 사용자 id, 아이템 id, rating이다.

# 마지막 10개만 출력
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [70]:
from surprise.model_selection import KFold

bsl_options = {
    'method': 'als',
    'n_epochs': 5,
    'reg_u': 12,
    'reg_i': 5
}
algo = surprise.BaselineOnly(bsl_options)

np.random.seed(0)
acc = np.zeros(3)
cv = KFold(3)
for i, (trainset, testset) in enumerate(cv.split(dataframe)):
    algo.fit(trainset)
    predictions = algo.test(testset)
    acc[i] = surprise.accuracy.rmse(predictions, verbose=True)
acc.mean()

Estimating biases using als...
RMSE: 0.9454
Estimating biases using als...
RMSE: 0.9377
Estimating biases using als...
RMSE: 0.9500


0.9443731335854068

In [71]:
from surprise.model_selection import cross_validate

cross_validate(algo, dataframe)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.93852765, 0.94652511, 0.93621515, 0.94228131, 0.9443244 ]),
 'test_mae': array([0.74497763, 0.75130899, 0.73987318, 0.7458679 , 0.74674833]),
 'fit_time': (0.09366917610168457,
  0.09369611740112305,
  0.09055066108703613,
  0.09372663497924805,
  0.1216742992401123),
 'test_time': (0.1269059181213379,
  0.07813453674316406,
  0.15621280670166016,
  0.0853111743927002,
  0.08278083801269531)}

#### 3. 추천성능 평가기준

아래 식에서  R헷은 테스트 데이터셋을 뜻한다.

<img src="4.jpg" width="1000" />

#### 4. Collaborative Filter 개요

<img src="5.jpg" width="1000" />

#### 5. collaborative filter를 위한 유사도 계산법

<img src="7.jpg" width="1000" />

5-1) 평균제곱 차이 유사도

<img src="8.jpg" width="1000" />

#### Neighborhood 모형

<img src="6.jpg" width="1000" />